In [ ]:
1.   Efetuar a média do campo Rating agrupando pelo campo Category
2.   Efetuar a média e o desvio padrão do campo Size agrupando por Genres (valores contidos no campo), ignorando registros onde o tamanho varia pelo device
3.   Efetuar a média do campo Reviews em janelas de 30 dias, agrupando pelo campo Category
4.   Capturar o email do atendimento que se encontra no final da página https://mmartan.com.br/ via Web Scraping e escreve-lo na tela


In [1]:
#importando a biblioteca do pandas (https://pandas.pydata.org/)
import pandas as pd

import numpy as np

from datetime import date


In [77]:
#metodo responsavel por realizar a leitura do arquivo
dados = 'C:/Users/guihb/Desktop/Coteminas/googleplaystore.csv'
df_coteminas = pd.read_csv(dados)

In [78]:
#Método para visualizar a estrutura
df_coteminas.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.2004,4.0.3 and up


In [79]:
#1.   Efetuar a média do campo Rating agrupando pelo campo Category
df_media = df_coteminas[['Rating','Category']].groupby(['Category'], as_index = False).mean()
df_media_final = round(df_media,3)
df_media_final

,Category,Rating
0,ART_AND_DESIGN,4.358
1,AUTO_AND_VEHICLES,4.190
2,BEAUTY,4.279
3,BOOKS_AND_REFERENCE,4.346
4,BUSINESS,4.121
...,...,...
28,SPORTS,4.224
29,TOOLS,4.044
30,TRAVEL_AND_LOCAL,4.109
31,VIDEO_PLAYERS,4.064


In [80]:
#2.   Efetuar a média e o desvio padrão do campo Size agrupando por Genres (valores contidos no campo), ignorando registros onde o tamanho varia pelo device

#Bom, primeiramente, eu ignorei os 'Varies with device' e trouxe as duas colunas necessárias.
df_media_size = df_coteminas[df_coteminas.Size != 'Varies with device'][['Genres','Size']]
# Nesta etapa, eu tive que modificar os Tamanhos em números para poder calcular a média.
df_media_size["Size"] = df_media_size["Size"].apply(lambda x: x.replace("M", "000000")).apply(lambda x: x.replace("k", "000")) 
df_media_size.head(4) 

,Genres,Size
0,Art & Design,19000000
1,Art & Design;Pretend Play,14000000
2,Art & Design,8.7000000
3,Art & Design,25000000


In [81]:
#Nesta etapa, tive que transformar os dados de 'Size' que são qualitativos para quantitativos
df_media_size['Size'] = pd.to_numeric(df_media_size.Size).astype(np.int64, errors='ignore')

In [82]:
#Nesta etapa, realizei a média dos Sizes agrupando por Genres.

media_size = df_media_size[df_media_size.Size != 'Varies with device'][['Genres','Size']].groupby(['Genres'], as_index = False).mean()
print(round(media_size,2))

                                    Genres         Size
0                                   Action  50164046.14
1                Action;Action & Adventure  53000000.00
2                                Adventure  36159421.14
3             Adventure;Action & Adventure  49692308.69
4                    Adventure;Brain Games         8.00
..                                     ...          ...
111                Video Players & Editors  13706002.07
112     Video Players & Editors;Creativity  23000000.00
113  Video Players & Editors;Music & Video  19000000.00
114                                Weather  10589160.23
115                                   Word  34480001.08

[116 rows x 2 columns]


In [83]:
#E por final, o desvio padrão Amostral
dpa_coteminas = media_size.std(ddof=1)
print('O desvio Padrão Amostral:', round(dpa_coteminas,2))

O desvio Padrão Amostral: Size    21567381.29
dtype: float64


In [84]:
#3.   Efetuar a média do campo Reviews em janelas de 30 dias, agrupando pelo campo Category

#Primeiramente mudei a coluna data de Str para Datetime
df_coteminas['Last Updated'] = df_coteminas['Last Updated'].astype('datetime64')
#para ficar uma imagem melhor de ser exibida, limitei os registros da seguinte manera:
pd.set_option('display.max_rows',10)

In [85]:
#Etapa importante, onde separo apenas os meses e os anos para trabalhar no range de 30 dias.
df_coteminas['Last Updated'] = df_coteminas['Last Updated'].map(lambda x: x.month + 100*x.year)

In [86]:
#Nesta etapa separei apenas os dados que o cliente queria
dados = df_coteminas[['Category','Rating','Last Updated']]
dados

,Category,Rating,Last Updated
0,ART_AND_DESIGN,4.1,201801
1,ART_AND_DESIGN,3.9,201801
2,ART_AND_DESIGN,4.7,201808
3,ART_AND_DESIGN,4.5,201806
4,ART_AND_DESIGN,4.3,201806
...,...,...,...
10836,FAMILY,4.5,201707
10837,FAMILY,5.0,201807
10838,MEDICAL,NaN,201701
10839,BOOKS_AND_REFERENCE,4.5,201501


In [87]:
# E por fim, Efetuo a média do campo Reviews em janelas de 30 dias, ou seja, por mês, agrupando pelo campo Category
media_review = dados.groupby([dados['Category'], 'Last Updated'])[['Rating']].mean().reset_index()
media_review

,Category,Last Updated,Rating
0,ART_AND_DESIGN,201703,4.400000
1,ART_AND_DESIGN,201705,4.200000
2,ART_AND_DESIGN,201708,4.100000
3,ART_AND_DESIGN,201709,4.400000
4,ART_AND_DESIGN,201710,4.550000
...,...,...,...
1167,WEATHER,201804,4.340000
1168,WEATHER,201805,4.400000
1169,WEATHER,201806,4.200000
1170,WEATHER,201807,4.277778


In [88]:
#4.   Capturar o email do atendimento que se encontra no final da página https://mmartan.com.br/ via Web Scraping e escreve-lo na tela
#Usei a aplicação chamada ParseHub para fazer o Web Scrapin do Email da Web mmartam.
email = 'C:/Users/guihb/Desktop/Coteminas/email_coteminas.csv'
email_coteminas = pd.read_csv(email)
email_coteminas

,Email_Desafio
0,atendimento@mmartan.com.br
